In [50]:
import pandas as pd
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'

db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [51]:
# please change esm.last_coding_date & am.facility_id = '314' in the query as per requirement
# am.facility_id is prod_id from go-live sheet
query = '''
SELECT 
    CONVERT_TZ(em.service_date, 'UTC', 'US/Central') AS service_date,
    GROUP_CONCAT(DISTINCT eandm.code) AS final,
    GROUP_CONCAT(DISTINCT dc.code) AS suggested,
    am.account_number AS account_number,
    facility_id,
    client_id
FROM
    account_mst am
        INNER JOIN
    document_mst dm ON dm.account_id = am.id
        AND dm.is_physician_document = 1
        AND dm.is_active = 1
        INNER JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    document_code dc ON dc.document_id = dm.id
        AND dc.is_active = 1
        AND dc.code_category = 'EANDM'
        INNER JOIN
    encounter_mst em ON em.account_id = dm.account_id
        AND em.service_date = dm.service_date
        AND em.physician_id = dm.physician_id
        AND em.is_active = 1
        INNER JOIN
    encounter_status_map esm ON esm.encounter_id = em.id
        AND esm.is_active = 1
        AND esm.encounter_status_id IN (9 , 17)
        INNER JOIN
    eandm_code eandm ON eandm.encounter_id = em.id
        AND eandm.process_id = esm.process_id
        AND eandm.user_id = esm.user_id
        AND eandm.is_active = 1
WHERE
dpd.id = (SELECT 
            id
        FROM
            document_processing_detail
        WHERE
            document_id = dm.id
        ORDER BY created_date DESC
        LIMIT 1)
        AND am.facility_id = '19'
        AND am.client_id = '14'
        AND esm.id = (SELECT 
            id
        FROM
            encounter_status_map
        WHERE
            encounter_id = em.id
                AND encounter_status_id IN (9 , 17)
        ORDER BY last_coding_date DESC
        LIMIT 1)
        AND am.is_active = 1
        AND CONVERT_TZ(esm.last_coding_date,
            'UTC',
            'US/Central') >= '2023-01-01'
GROUP BY am.account_number , em.physician_id , em.service_date , dm.id
'''
        # AND CONVERT_TZ(em.service_date,
        #     'UTC',
        #     'US/Central') > '2023-03-15'
        
        #     -- am.account_number = 8594135

In [52]:
df = pd.read_sql(query, con=db_connection)

In [53]:
df.facility_id.unique()

array([19])

In [54]:
df = df[['service_date', 'account_number', 'suggested', 'final']]

In [47]:
df

,service_date,account_number,suggested,final
0,2023-05-22 08:00:00,8056308,99214,99214
1,2023-05-22 08:30:00,8056310,99214,99214
2,2023-05-23 10:00:00,8056352,99214,99214
3,2023-05-23 11:00:00,8056356,99213,99213
4,2023-05-23 13:00:00,8056364,99214,99214
...,...,...,...,...
79,2023-05-30 10:30:00,8594285,99203,99203
80,2023-05-24 08:00:00,8749413,99214,99214
81,2023-05-23 10:30:00,8749591,99204,99204
82,2023-05-31 10:00:00,8750591,99214,99214


In [48]:
# check for nan in final and suggested columns

In [29]:
df.shape

(9596, 4)

In [30]:
def get_counts(x):
    
    suggested = str(x['suggested'])
    final = str(x['final'])
    
    suggested_set = set(suggested.split(','))   
    final_set = set(final.split(','))
    
    accepted = len(suggested_set.intersection(final_set))
    rejected = len(suggested_set - final_set)
    added = len(final_set - suggested_set)
    if (rejected >= 1 or added >= 1):
        print('rejected',rejected)
        print('added',added)
        print('suggested_set',suggested_set,'\t final_set',final_set)
        print('accountNumber', str(x['account_number']))
            
    return pd.Series({'accepted':accepted, 'rejected':rejected, 'added':added})

In [31]:
count_df =  df.apply(get_counts, axis=1)

rejected 1
added 1
suggested_set {'99282'} 	 final_set {'99283'}
accountNumber H00021184064
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99283'}
accountNumber H00021184073
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99283'}
accountNumber H00021184119
rejected 0
added 1
suggested_set {'99284'} 	 final_set {'99284', '99283'}
accountNumber H00021184172
rejected 1
added 1
suggested_set {'99282'} 	 final_set {'99284'}
accountNumber H00021184239
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99283'}
accountNumber H00021184263
rejected 0
added 1
suggested_set {'99283'} 	 final_set {'99284', '99283'}
accountNumber H00021184336
rejected 0
added 1
suggested_set {'99284'} 	 final_set {'99284', '99285'}
accountNumber H00021184486
rejected 0
added 1
suggested_set {'99285'} 	 final_set {'99284', '99285'}
accountNumber H00021184498
rejected 0
added 1
suggested_set {'99284'} 	 final_set {'99284', '99285'}
accountNumber H00021184500
rejected 1
added 1
suggested_set {'

In [32]:
count_df

,accepted,rejected,added
0,1,0,0
1,0,1,1
2,0,1,1
3,0,1,1
4,1,0,0
...,...,...,...
9591,1,0,0
9592,1,0,0
9593,0,1,1
9594,1,0,0


In [33]:
count_df.accepted.sum(), count_df.rejected.sum(), count_df.added.sum()

(6895, 2978, 3183)

In [34]:
precision = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.rejected.sum())

In [35]:
recall = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.added.sum())

In [36]:
f1_score = (2 * precision * recall) / (recall + precision)

In [37]:
precision, recall, f1_score

(0.6983692899827814, 0.6841635245088311, 0.6911934238885269)